In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)
print(y_pred)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 105us/sample - loss: 0.6877 - val_loss: 7.2633
Epoch 2/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4525 - val_loss: 1.3337
Epoch 3/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.3976 - val_loss: 0.7138
Epoch 4/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.3843 - val_loss: 0.4327
Epoch 5/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.3784 - val_loss: 0.4054
Epoch 6/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.3706 - val_loss: 0.4030
Epoch 7/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.3691 - val_loss: 1.5574
Epoch 8/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4841 - val_loss: 0.9902
Epoch 9/20
11610/11610 [==============================] - 0s 27us/sample - loss: 0.3914 - val_

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 0s 41us/sample - loss: 1.3082 - val_loss: 0.7809
Epoch 2/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.4741 - val_loss: 0.4703
Epoch 3/20
11610/11610 [==============================] - 0s 31us/sample - loss: 0.4053 - val_loss: 0.4069
Epoch 4/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3827 - val_loss: 0.3905
Epoch 5/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3717 - val_loss: 0.3878
Epoch 6/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3671 - val_loss: 0.3958
Epoch 7/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3576 - val_loss: 0.3965
Epoch 8/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3553 - val_loss: 0.3808
Epoch 9/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.3511 - val_l

In [7]:
model3.layers

In [8]:
model3.layers[1]

In [9]:
hidden1 = model3.layers[1]

In [10]:
hidden1.name

'dense_8'

In [11]:
model3.get_layer('dense_8') is hidden1

True